In [6]:
import pandas as pd #For reading dataset files
import networkx as nx #For network creation/analysis
from networkx.algorithms import community
import community as community_louvain
import matplotlib.pyplot as plt #For plotting graphs
import igraph as ig
import numpy as np
import leidenalg as la
from sklearn.metrics import (
    adjusted_rand_score,
    adjusted_mutual_info_score,
    normalized_mutual_info_score,
    homogeneity_score,
    completeness_score,
    f1_score,
    confusion_matrix,
)
from collections import Counter
from networkx.algorithms.community import label_propagation_communities
from cdlib import algorithms

from modularitydensity.metrics import modularity_density
import community as louvain
import leidenalg as la
from cdlib.algorithms import louvain

from cdlib import evaluation

import os

import random

import dask.array as da
from sklearn.metrics import confusion_matrix
from scipy.sparse import coo_matrix

np.random.seed(42)  # Imposta il seed per NumPy
random.seed(42)

In [7]:
from cdlib import datasets
graph_name_list = datasets.available_ground_truths()
print(graph_name_list)

['karate_club', 'dblp', 'amazon', 'youtube', 'LFR_N100000_ad5_mc50_mu0.1', 'LFR_N100000_ad5_mc50_mu0.2', 'LFR_N100000_ad5_mc50_mu0.3', 'LFR_N100000_ad5_mc50_mu0.4', 'LFR_N100000_ad5_mc50_mu0.5', 'LFR_N100000_ad5_mc50_mu0.6', 'LFR_N100000_ad5_mc50_mu0.7', 'LFR_N100000_ad5_mc50_mu0.8', 'LFR_N100000_ad5_mc50_mu0.9', 'LFR_N10000_ad5_mc50_mu0.1', 'LFR_N10000_ad5_mc50_mu0.2', 'LFR_N10000_ad5_mc50_mu0.3', 'LFR_N10000_ad5_mc50_mu0.4', 'LFR_N10000_ad5_mc50_mu0.5', 'LFR_N10000_ad5_mc50_mu0.6', 'LFR_N10000_ad5_mc50_mu0.7', 'LFR_N10000_ad5_mc50_mu0.8', 'LFR_N10000_ad5_mc50_mu0.9', 'LFR_N1000_ad5_mc50_mu0.1', 'LFR_N1000_ad5_mc50_mu0.2', 'LFR_N1000_ad5_mc50_mu0.3', 'LFR_N1000_ad5_mc50_mu0.4', 'LFR_N1000_ad5_mc50_mu0.5', 'LFR_N1000_ad5_mc50_mu0.6', 'LFR_N1000_ad5_mc50_mu0.7', 'LFR_N1000_ad5_mc50_mu0.8', 'LFR_N1000_ad5_mc50_mu0.9', 'LFR_N50000_ad5_mc50_mu0.1', 'LFR_N50000_ad5_mc50_mu0.2', 'LFR_N50000_ad5_mc50_mu0.3', 'LFR_N50000_ad5_mc50_mu0.4', 'LFR_N50000_ad5_mc50_mu0.5', 'LFR_N50000_ad5_mc50_mu0.6'

# Login dataset

In [8]:
from cdlib import datasets

G = datasets.fetch_network_data(net_name="LFR_N100000_ad5_mc50_mu0.1", net_type="igraph")

gt_coms = datasets.fetch_ground_truth_data(net_name="LFR_N100000_ad5_mc50_mu0.1")

# Converti la ground truth in un dizionario con nodi e cluster
communities_dict = {}
for community_id, community in enumerate(gt_coms.communities):
    communities_dict[community_id] = community

# Stampa il numero di nodi e di comunità
print("Numero di nodi nel grafo:", len(G.vs))
print("Numero di comunità nella ground truth:", len(gt_coms.communities))

# Stampa la ground truth nel formato richiesto
print("Ground truth delle comunità:", communities_dict)


Numero di nodi nel grafo: 100000
Numero di comunità nella ground truth: 585
Ground truth delle comunità: {0: [22539, 94219, 34833, 88084, 38932, 22555, 51228, 32797, 98336, 84005, 43, 49204, 6196, 63545, 81982, 86081, 71749, 41029, 41031, 45126, 57417, 90187, 67660, 49229, 32845, 30800, 28753, 94290, 69717, 34912, 53351, 41075, 30836, 34939, 71815, 43152, 94353, 59537, 12433, 86170, 45219, 41129, 37035, 12459, 12463, 82096, 78005, 59577, 28862, 6334, 88254, 65735, 78024, 37072, 69845, 57558, 69847, 30934, 26843, 28891, 37088, 45286, 10470, 33003, 96495, 49392, 30960, 51447, 78072, 57595, 51452, 39165, 35071, 8448, 55553, 78083, 90371, 69894, 8454, 67848, 22794, 26894, 49425, 71958, 26904, 61723, 18716, 45341, 78111, 10529, 78114, 24865, 98598, 41260, 96557, 78127, 59695, 90417, 8500, 59703, 86328, 6457, 26937, 18761, 47433, 49483, 59723, 67918, 59730, 92499, 45396, 35159, 51551, 39270, 86377, 96623, 18799, 53619, 20855, 90503, 16781, 37263, 22930, 76187, 22940, 47517, 82334, 80283, 415

In [9]:
def calcola_metriche(comunita_attese, comunita_rilevate, adjacency_matrix):
    # Creazione della mappatura dei nodi a indici
    nodi = []
    for community in comunita_attese.values():
        nodi.extend(community)
    nodo_to_index = {nodo: idx for idx, nodo in enumerate(nodi)}

    # Preparazione dei dati per ARI e NMI
    true_labels = np.zeros(len(nodi), dtype=int)  # Etichette vere
    for community_id, community_nodes in comunita_attese.items():
        for node in community_nodes:
            true_labels[nodo_to_index[node]] = int(community_id)  # Usa l'ID della comunità

    predicted_labels = np.full(len(nodi), -1)  # Valori non assegnati    
    for cluster_id, nodes in comunita_rilevate.items():
        for node in nodes:
            if node in nodo_to_index:  # Assicurati che il nodo esista nella mappatura
                predicted_labels[nodo_to_index[node]] = int(cluster_id)  # Usa l'ID della comunità

    # Calcolo delle metriche
    ari = adjusted_rand_score(true_labels, predicted_labels)
    ami = adjusted_mutual_info_score(true_labels, predicted_labels)
    nmi = normalized_mutual_info_score(true_labels, predicted_labels)

    # Calcolo dell'HS
    hs = homogeneity_score(true_labels, predicted_labels)

    # Calcolo del CS
    cs = completeness_score(true_labels, predicted_labels)

    # Calcolo della F1 Score
    f1 = f1_score(true_labels, predicted_labels, average='weighted')

    # Calcolo della Confusion Matrix
    conf_matrix = confusion_matrix(true_labels, predicted_labels)

    # Calcolo della Modularity Density
    md = modularity_density(adjacency_matrix, comunita_rilevate, nodo_to_index)

    # Calcolo della Community Score
    cs_score = community_score(adjacency_matrix, comunita_rilevate, nodo_to_index)

    # Calcolo della Community Fitness
    fitness = community_fitness(adjacency_matrix, comunita_rilevate, nodo_to_index)

    # Stampa dei risultati
    print(f"Adjusted Rand Index (ARI): {ari:.4f}")
    print(f"Adjusted Mutual Information (AMI): {ami:.4f}")
    print(f"Homogeneity Score (HS): {hs:.4f}")
    print(f"Completeness Score (CS): {cs:.4f}")
    print(f"F1 Score: {f1:.4f}")
    print(f"Confusion Matrix:\n{conf_matrix}")
    print(f"Modularity Density: {md:.4f}")
    print(f"Community Score: {cs_score:.4f}")
    print(f"Community Fitness: {fitness:.4f}")
    print(f"Normalized Mutual Information (NMI): {nmi:.4f}")

    # Risultati da visualizzare
    metriche = {
        'ARI': ari,
        'AMI': ami,
        'HS': hs,
        'CS': cs,
        'F1 Score': f1,
        'Modularity Density': md,
        'Community Score': cs_score,
        'Community Fitness': fitness,
        'NMI': nmi
    }

    return metriche

    # Creazione del grafico a barre
    plt.figure(figsize=(10, 6))
    plt.bar(metriche.keys(), metriche.values(), color='skyblue')
    plt.ylabel('Valore')
    plt.title('Metriche di Rilevamento delle Comunità')
    plt.xticks(rotation=45)
    plt.ylim(0, 1)
    plt.grid(axis='y')
    plt.tight_layout()
    plt.show()

    x = np.arange(len(labels))
    bar_width = 0.35

    plt.figure(figsize=(10, 6))
    plt.bar(x - bar_width / 2, true_values, width=bar_width, label='Comunità Attese', color='skyblue')
    plt.bar(x + bar_width / 2, predicted_values, width=bar_width, label='Comunità Rilevate', color='salmon')
    plt.xlabel('Comunità')
    plt.ylabel('Numero di Nodi')
    plt.title('Confronto tra Comunità Attese e Rilevate')
    plt.xticks(x, labels)
    plt.legend()
    plt.grid(axis='y')
    plt.tight_layout()
    plt.show()

# Funzioni di supporto 
def modularity_density(adjacency_matrix, comunita_rilevate, nodo_to_index):
    k = len(comunita_rilevate)  # Numero di comunità
    densita = 0.0  # Inizializza la densità a zero

    for community in comunita_rilevate.values():
        V_c = len(community)  # Numero di nodi nella comunità
        E_c = 0  # Inizializza il conteggio degli archi nella comunità

        # Calcola il numero di archi tra i nodi nella comunità
        for i in range(V_c):
            for j in range(i + 1, V_c):  # Evita il conteggio doppio
                idx_i = nodo_to_index[community[i]]
                idx_j = nodo_to_index[community[j]]
                if adjacency_matrix[idx_i, idx_j] > 0:
                    E_c += 1

        # Calcola la densità della comunità
        if V_c > 1:  # Evita divisioni per zero
            densita += E_c / (V_c * (V_c - 1))

    # Calcola la densità modulare finale
    if k > 0:  # Evita divisioni per zero
        densita_modulare = densita / k
    else:
        densita_modulare = 0

    return densita_modulare

def community_score(adjacency_matrix, comunita_rilevate, nodo_to_index):
    internal_edges = 0
    total_edges = 0
    num_nodes = adjacency_matrix.shape[0]

    for community in comunita_rilevate.values():
        for i in range(len(community)):
            for j in range(i + 1, len(community)):  # Per evitare il conteggio doppio
                node_i = community[i]
                node_j = community[j]
                if adjacency_matrix[nodo_to_index[node_i], nodo_to_index[node_j]] > 0:
                    internal_edges += 1

    # Conta anche gli archi totali nel grafo
    total_edges = np.sum(adjacency_matrix) / 2  # Dato che è una matrice simmetrica

    if total_edges == 0:
        return 0  # Evita divisioni per zero
    return internal_edges / total_edges


def community_fitness(adjacency_matrix, comunita_rilevate, nodo_to_index):
    internal_edges = 0
    external_edges = 0
    num_nodes = adjacency_matrix.shape[0]  # Assicurati che la matrice sia quadrata

    node_to_community = {}
    for community_id, nodes in comunita_rilevate.items():
        for node in nodes:
            node_to_community[node] = community_id

    for i in range(num_nodes):
        for j in range(num_nodes):
            if adjacency_matrix[i, j] > 0:  # Se esiste un arco
                node_i = list(nodo_to_index.keys())[list(nodo_to_index.values()).index(i)]
                node_j = list(nodo_to_index.keys())[list(nodo_to_index.values()).index(j)]
                if node_to_community.get(node_i) == node_to_community.get(node_j):  # Se appartengono alla stessa comunità
                    internal_edges += 1
                else:
                    external_edges += 1

    total_edges = internal_edges + external_edges

    # Evita divisioni per zero
    if total_edges == 0:
        return 0

    return internal_edges / total_edges

# Label propagation

In [10]:
# Convertiamo il grafo da igraph a networkx
G_nx = nx.Graph(G.to_networkx())

# Rilevamento delle comunità con l'algoritmo Label Propagation
communities = algorithms.label_propagation(G_nx).communities
community_dict_label_propagation = {i: community for i, community in enumerate(communities)}

# Stampa delle comunità rilevate
for i, community in enumerate(communities):
    print(f"Community {i + 1}: {community}")


# Calcolo della matrice di adiacenza in formato sparso
adj_matrix_full = nx.adjacency_matrix(G_nx).todense()
# Esegui la funzione per calcolare metriche (aggiornala per accettare matrici sparse se necessario)
risultati_label_propagation = calcola_metriche(communities_dict, community_dict_label_propagation, adj_matrix_full)


Community 1: [48450, 48451, 38286, 38287, 55472, 4296, 4297, 4298, 4299, 32974, 32975, 32976, 32977, 32978, 32979, 24807, 24808, 24809, 22813, 22814, 22815, 22816, 35139, 29056, 29057, 29058, 29059, 29060, 29061, 31104, 31105, 31106, 70068, 20932, 38361, 25076, 25077, 25078, 37367, 37368, 25079, 37365, 37366, 25091, 25092, 78368, 31292, 31293, 25200, 25201, 25202, 38388, 17017, 17018, 17019, 17020, 76413, 80518, 38392, 35477, 35478, 35479, 70339, 33481, 33482, 47830, 38412, 31517, 31518, 31519, 80709, 73441, 27546, 29596, 29597, 29598, 29599, 33702, 33703, 33704, 33705, 33706, 25526, 25527, 76732, 969, 38461, 38465, 38466, 38467, 33798, 76824, 19513, 19514, 19515, 19516, 19517, 19518, 19519, 5225, 5226, 5227, 38489, 38490, 60879, 56451, 38037, 38038, 38040, 29880, 29881, 29882, 29883, 58906, 3294, 60641, 34061, 34062, 34063, 34064, 23832, 23833, 23834, 1313, 1314, 1315, 1316, 1317, 1318, 1319, 1320, 1321, 52773, 34101, 34102, 34103, 38206, 38207, 38208, 38209, 38210, 38211, 38212, 3821

MemoryError: Unable to allocate 74.5 GiB for an array with shape (100000, 100000) and data type int64

# Louvain

In [ ]:
np.random.seed(43)
random.seed(43)

G_nx = nx.Graph(G.to_networkx())
pos = nx.spring_layout(G_nx)

communities = algorithms.louvain(G_nx)  # Usa cdlib per rilevare le comunità
community_dict_louvain = {i: community for i, community in enumerate(communities.communities)}

# Stampa le comunità
for i, community in enumerate(communities.communities):
    print(f"Community {i + 1}: {community}")

# Converte in lista se necessario
community_list = list(communities.communities)

# Calcola la matrice di adiacenza completa
adj_matrix_full = nx.adjacency_matrix(G_nx).todense()

# Supponendo che calcola_metriche sia definito altrove
risultati_louvain = calcola_metriche(communities_dict, community_dict_louvain, adj_matrix_full)

# Leiden

In [ ]:
np.random.seed(43)
random.seed(43)

G_nx = nx.Graph(G.to_networkx())

pos = nx.spring_layout(G_nx)

communities = algorithms.leiden(G_nx).communities
community_dict_leiden = {i: community for i, community in enumerate(communities)}

for i, community in enumerate(communities):
    print(f"Community {i + 1}: {community}")

community_list=list(communities)

#Calcola le metriche
adj_matrix_full = nx.adjacency_matrix(G_nx).todense()
risultati_leiden = calcola_metriche(communities_dict, community_dict_leiden, adj_matrix_full)


# Risultati 

In [ ]:
# Aggiungi il numero delle comunità rilevate per ciascun algoritmo nei risultati
risultati = {
    "Label Propagation": {**risultati_label_propagation, "Numero di Comunità": len(community_dict_label_propagation)},
    "Louvain": {**risultati_louvain, "Numero di Comunità": len(community_dict_louvain)},
    "Leiden": {**risultati_leiden, "Numero di Comunità": len(community_dict_leiden)},
}

# Creazione di un DataFrame dalle metriche ottenute per ogni algoritmo, inclusi il numero di comunità
df_risultati = pd.DataFrame(risultati)

# Visualizzazione del DataFrame per vedere i risultati
print(df_risultati)

# Imposta dimensioni della figura per il grafico
fig, ax = plt.subplots(figsize=(10, 7))

# Definire le posizioni delle barre
n_misure = len(df_risultati.index)
bar_width = 0.2
index = np.arange(n_misure)

# Plot delle barre
for i, algoritmo in enumerate(df_risultati.columns):
    ax.bar(index + i * bar_width, df_risultati[algoritmo], bar_width, label=algoritmo)

# Aggiungere etichette e titolo
ax.set_xlabel('Metriche')
ax.set_ylabel('Valori')
ax.set_title('Metriche di valutazione per diversi algoritmi di clustering')

# Configurazione delle etichette per l'asse X
ax.set_xticks(index + bar_width * 1.5)
ax.set_xticklabels(df_risultati.index, rotation=45, ha='right')

# Aggiungere legenda
ax.legend()

# Mostrare il grafico
plt.tight_layout()
plt.show()


# Conga

In [ ]:
# Run the CONGA algorithm
G_nx = nx.Graph(G.to_networkx())
com = algorithms.conga(G_nx, number_communities=585)
community_dict = {i: community for i, community in enumerate(com.communities)}

# Print the detected communities
print("Communities detected:")
for i, community in enumerate(com.communities):
    print(f"Communità {i + 1}: {community}")



# Calcola le metriche
adj_matrix_full = nx.adjacency_matrix(G_nx).todense()
risultati_conga = calcola_metriche(communities_dict, community_dict, adj_matrix_full)